# Pandas vs DuckDB: A Chess Dataset Performance Comparison

### Introduction
In this notebook, we will compare **Pandas**, a widely-used Python library, with **DuckDB**, an efficient SQL-based analytical database, to analyze a large chess dataset. 

As datasets grow in size, choosing the right tool becomes critical for performance and scalability. Through this comparison, we aim to highlight the strengths and trade-offs of each tool.


### Introduction to Pandas and DuckDB

- **Pandas**:
  - Pandas is a popular **Python library** for data analysis and manipulation.
  - It provides **DataFrames**, which are easy to use for tasks like filtering, sorting, and aggregating data.
  - However, Pandas **loads the entire dataset into memory**, which makes it inefficient for large datasets.

- **DuckDB**:
  - DuckDB is a **SQL-based analytical database** designed to process large datasets efficiently.
  - It works seamlessly with Pandas DataFrames but uses SQL queries for operations.
  - DuckDB’s **columnar storage** and query optimizations make it much faster for big data tasks.


### Why Compare Pandas and DuckDB?

As datasets grow larger—like the chess dataset with over **6 million rows**—memory efficiency and query performance become critical. Comparing these tools reveals their **strengths and trade-offs**, helping us understand:
- Where Pandas excels (e.g., small-scale, Python-native tasks).
- Where DuckDB dominates (e.g., analytical workloads, large datasets).

### Objective
By comparing Pandas and DuckDB on the chess dataset, we aim to:
- Highlight **performance differences** for common data operations like filtering, aggregation, and sorting.
- Understand **why DuckDB performs better for large datasets** while Pandas remains a strong choice for simpler tasks.


### Understanding Row-Based vs Columnar Storage

The primary difference between Pandas and DuckDB lies in how they store and access data:

1. **Pandas (Row-Based Storage)**:
   - Data is stored row by row.
   - When accessing data from a specific column, Pandas still loads the entire row into memory, leading to inefficiencies for large datasets.

2. **DuckDB (Columnar Storage)**:
   - Data is stored column by column.
   - DuckDB reads only the relevant columns for a query, minimizing memory usage and improving speed.

Here’s a diagram explaining these differences:

![Row vs Column Storage](pandas_duckdb.png)


### Chess Dataset Overview

This dataset contains information about chess games, including:
- **Event**: Type of game (e.g., Classical, Blitz).
- **WhiteElo** and **BlackElo**: Player ratings.
- **Opening**: The opening strategy used in the game.
- **Result**: Outcome of the game.

We chose this dataset because:
- It contains over **6 million rows**, making it ideal for testing performance differences.
- It represents real-world data, relatable and diverse enough to explore multiple data operations.


In [32]:
import pandas as pd

# Load the dataset
file_path = 'chess_games.csv'
df = pd.read_csv(file_path)

# Display dataset preview
print("Dataset Preview:")
df.head()


Dataset Preview:


,Event,White,Black,Result,UTCDate,UTCTime,WhiteElo,BlackElo,WhiteRatingDiff,BlackRatingDiff,ECO,Opening,TimeControl,Termination,AN
0,Classical,eisaaaa,HAMID449,1-0,2016.06.30,22:00:01,1901,1896,11.0,-11.0,D10,Slav Defense,300+5,Time forfeit,1. d4 d5 2. c4 c6 3. e3 a6 4. Nf3 e5 5. cxd5 e...
1,Blitz,go4jas,Sergei1973,0-1,2016.06.30,22:00:01,1641,1627,-11.0,12.0,C20,King's Pawn Opening: 2.b3,300+0,Normal,1. e4 e5 2. b3 Nf6 3. Bb2 Nc6 4. Nf3 d6 5. d3 ...
2,Blitz tournament,Evangelistaizac,kafune,1-0,2016.06.30,22:00:02,1647,1688,13.0,-13.0,B01,Scandinavian Defense: Mieses-Kotroc Variation,180+0,Time forfeit,1. e4 d5 2. exd5 Qxd5 3. Nf3 Bg4 4. Be2 Nf6 5....
3,Correspondence,Jvayne,Wsjvayne,1-0,2016.06.30,22:00:02,1706,1317,27.0,-25.0,A00,Van't Kruijs Opening,-,Normal,1. e3 Nf6 2. Bc4 d6 3. e4 e6 4. Nf3 Nxe4 5. Nd...
4,Blitz tournament,kyoday,BrettDale,0-1,2016.06.30,22:00:02,1945,1900,-14.0,13.0,B90,"Sicilian Defense: Najdorf, Lipnitsky Attack",180+0,Time forfeit,1. e4 c5 2. Nf3 d6 3. d4 cxd4 4. Nxd4 Nf6 5. N...


### Dataset Size
Let’s check the number of rows and columns in the dataset. This gives us an idea of the data scale and why performance differences between Pandas and DuckDB matter.


In [33]:
# Display the shape of the dataset
print(f"The dataset contains {df.shape[0]} rows and {df.shape[1]} columns.")


The dataset contains 6256184 rows and 15 columns.


# Joining Tables

### Example 1: Joining tables
Task: Perform a self-join to match games with the same Opening and calculate the average WhiteElo difference between the matched games..

In [34]:
import time

# Measure execution time
def time_execution(func, label):
    start_time = time.time()
    result = func()
    elapsed_time = time.time() - start_time
    print(f"{label}: {elapsed_time:.4f} seconds")
    return result ,elapsed_time

# Pandas Filtering
def pandas_join():
    joined = df.merge(df, on='Opening', suffixes=('_1', '_2'))
    joined['EloDiff'] = abs(joined['WhiteElo_1'] - joined['WhiteElo_2'])
    return joined[['Opening', 'EloDiff']].head()

pandas_joined = time_execution(pandas_join, "Pandas Self-Join")
        


MemoryError: Unable to allocate 1.35 TiB for an array with shape (185247197766,) and data type int64

#### **Why Pandas Failed**
1. **Cartesian Explosion**:
   - A self-join creates all possible combinations of rows with the same `Opening`.
   - For a dataset with millions of rows, this results in an **exponentially larger intermediate DataFrame**, which becomes too large to fit into memory.

2. **Memory Limitations**:
   - Pandas attempts to load the entire intermediate result into memory.
   - In this case, Pandas tried to allocate **1.35 TiB** of memory for the operation, which exceeds typical system limits, causing a **MemoryError**.

3. **Row-Based Processing**:
   - Pandas processes rows inefficiently for such large-scale operations, compounding the memory and execution time issues.

4. **No Query Optimization**:
   - Pandas doesn’t optimize join operations, so it processes the entire DataFrame without applying early filters or memory-saving techniques.



In [35]:
import duckdb

def duckdb_join():
    con = duckdb.connect()
    con.register("df_table", df)  # Register the DataFrame
    query = """
    SELECT a.Opening, ABS(a.WhiteElo - b.WhiteElo) AS EloDiff
    FROM df_table a
    INNER JOIN df_table b
    ON a.Opening = b.Opening
    LIMIT 10
    """
    return con.execute(query).fetch_df()

duckdb_joined, duckdb_time = time_execution(duckdb_join, "DuckDB Self-Join")

print("Results of DuckDB Self-Join (Sample):")
display(duckdb_joined)  # Displaying as a formatted table for Jupyter Notebook

DuckDB Self-Join: 0.6482 seconds
Results of DuckDB Self-Join (Sample):


,Opening,EloDiff
0,"Ruy Lopez: Exchange Variation, Gligoric Variation",388
1,Giuoco Piano,230
2,Van Geet Opening,401
3,"Sicilian Defense: Modern Variations, Tartakower",103
4,Petrov: Modern Attack,589
5,"Sicilian Defense: Najdorf, Lipnitsky Attack",76
6,English Defense #2,45
7,Queen's Pawn Game: Krause Variation,49
8,King's Gambit,395
9,Rat Defense: Small Center Defense,259


In [36]:
import pandas as pd
from IPython.display import display

# Define the execution times
results_data = {
    "Tool": ["Pandas", "DuckDB"],
    "Execution Time (seconds)": [None, duckdb_time]  # Pandas exceeds time limit
}

# Create a DataFrame for display
comparison_table = pd.DataFrame(results_data)

# Display the comparison table
display(comparison_table)


,Tool,Execution Time (seconds)
0,Pandas,NaN
1,DuckDB,0.64816


#### **Why DuckDB Succeeded**
1. **Optimized Query Execution**:
   - DuckDB uses **chunk-based processing**, processing smaller parts of the data in memory without loading the entire result at once.

2. **Columnar Storage**:
   - DuckDB reads only the relevant columns (`Opening`, `WhiteElo`) for the join operation.
   - This significantly reduces the memory footprint compared to Pandas.

3. **Query Plan Optimization**:
   - DuckDB applies query optimizations like:
     - **Predicate Pushdown**: Applies the `LIMIT 10` filter early in the query execution to avoid unnecessary computations.
     - **Efficient Join Algorithms**: Uses optimized join techniques to reduce computational overhead.

4. **Scalability**:
   - DuckDB is designed to handle large datasets and complex analytical workloads efficiently, making it a natural choice for operations like self-joins.

---

# Aggregation

### Example 2: Aggregation
Task: Calculate the **average ELO ratings** for each opening.

#### Key Question:
- Which tool handles grouping and aggregations faster?


In [47]:
# Pandas Aggregation
def pandas_aggregation():
    return df.groupby('Opening')[['WhiteElo', 'BlackElo']].mean()

pandas_avg_elo , pandas_agg_time = time_execution(pandas_aggregation, "Pandas Aggregation")


Pandas Aggregation: 0.9558 seconds


In [50]:
# DuckDB Aggregation
def duckdb_aggregation():
    con = duckdb.connect()
    query = """
    SELECT Opening, AVG(WhiteElo) AS Avg_WhiteElo, AVG(BlackElo) AS Avg_BlackElo
    FROM df
    GROUP BY Opening
    """
    return con.execute(query).fetch_df()

duckdb_avg_elo , duckdb_agg_time = time_execution(duckdb_aggregation, "DuckDB Aggregation")
display( duckdb_avg_elo )

DuckDB Aggregation: 0.1322 seconds


,Opening,Avg_WhiteElo,Avg_BlackElo
0,Ruy Lopez: Berlin Defense #2,1657.154551,1647.638276
1,Dutch Defense: Rubinstein Variation,1864.678308,1870.287337
2,Vienna Game: Vienna Gambit,1864.451350,1789.471727
3,Scotch Game: Haxo Gambit,1816.674622,1788.013388
4,Sicilian Defense: Chekhover Variation,1834.700628,1842.042801
...,...,...,...
2937,Blackmar-Diemer Gambit: Rook Pawn Defense,2193.000000,1951.000000
2938,"Ruy Lopez: Open Variations, Malkin Variation",2290.000000,2206.000000
2939,"Sicilian Defense: Dragon, Classical, Zollner G...",1924.000000,2258.000000
2940,"Gruenfeld Defense: Exchange Variation, Larsen ...",1911.000000,1880.000000


In [39]:
import pandas as pd
from IPython.display import display

# Define the execution times
results_data = {
    "Tool": ["Pandas", "DuckDB"],
    "Execution Time (seconds)": [pandas_agg_time, duckdb_agg_time]  # Pandas exceeds time limit
}

# Create a DataFrame for display
comparison_table = pd.DataFrame(results_data)

# Display the comparison table
display(comparison_table)


,Tool,Execution Time (seconds)
0,Pandas,0.928950
1,DuckDB,0.137369


# Sorting Data

### Example 3: Sorting Data
Task: Identify the **top 10 most frequent chess openings**.

#### Key Question:
- Which tool sorts data faster?


In [40]:
# Pandas Sorting
def pandas_sorting():
    return df['Opening'].value_counts().head(10)

pandas_top_openings , pandas_sort_time = time_execution(pandas_sorting, "Pandas Sorting")
print(pandas_top_openings)

Pandas Sorting: 0.6462 seconds
Opening
Van't Kruijs Opening                             133112
Scandinavian Defense: Mieses-Kotroc Variation    112227
Modern Defense                                   108120
Horwitz Defense                                   95450
Sicilian Defense                                  85645
French Defense: Knight Variation                  83519
Caro-Kann Defense                                 82408
Scandinavian Defense                              78494
Owen Defense                                      73452
Sicilian Defense: Bowdler Attack                  72457
Name: count, dtype: int64


In [41]:
# DuckDB Sorting
def duckdb_sorting():
    con = duckdb.connect()
    query = """
    SELECT Opening, COUNT(*) AS Frequency
    FROM df
    GROUP BY Opening
    ORDER BY Frequency DESC
    LIMIT 10
    """
    return con.execute(query).fetch_df()

duckdb_top_openings , duckdb_sort_time = time_execution(duckdb_sorting, "DuckDB Sorting")
display(duckdb_top_openings)

DuckDB Sorting: 0.1510 seconds


,Opening,Frequency
0,Van't Kruijs Opening,133112
1,Scandinavian Defense: Mieses-Kotroc Variation,112227
2,Modern Defense,108120
3,Horwitz Defense,95450
4,Sicilian Defense,85645
5,French Defense: Knight Variation,83519
6,Caro-Kann Defense,82408
7,Scandinavian Defense,78494
8,Owen Defense,73452
9,Sicilian Defense: Bowdler Attack,72457


In [42]:
import pandas as pd
from IPython.display import display

# Define the execution times
results_data = {
    "Tool": ["Pandas", "DuckDB"],
    "Execution Time (seconds)": [pandas_sort_time, duckdb_sort_time]  # Pandas exceeds time limit
}

# Create a DataFrame for display
comparison_table = pd.DataFrame(results_data)

# Display the comparison table
display(comparison_table)


,Tool,Execution Time (seconds)
0,Pandas,0.646200
1,DuckDB,0.151029


# Custom Calculations

### Example 4: Custom Calculations
Task: Calculate the **absolute difference in ELO ratings** between players.

#### Key Question:
- How do the tools handle custom Python-like operations?


In [43]:
# Pandas Custom Calculation
def pandas_custom_calc():
    df['EloDifference'] = abs(df['WhiteElo'] - df['BlackElo'])
    return df[['WhiteElo', 'BlackElo', 'EloDifference']].head()

pandas_elo_diff , pandas_custom_time= time_execution(pandas_custom_calc, "Pandas Custom Calculation")
print(pandas_elo_diff)

Pandas Custom Calculation: 0.1991 seconds
   WhiteElo  BlackElo  EloDifference
0      1901      1896              5
1      1641      1627             14
2      1647      1688             41
3      1706      1317            389
4      1945      1900             45


In [45]:
# DuckDB Custom Calculation
def duckdb_custom_calc():
    con = duckdb.connect()
    query = """
    SELECT WhiteElo, BlackElo, ABS(WhiteElo - BlackElo) AS EloDifference
    FROM df
    """
    return con.execute(query).fetch_df().head()

duckdb_elo_diff , duckdb_custom_time = time_execution(duckdb_custom_calc, "DuckDB Custom Calculation")
display(duckdb_elo_diff)

DuckDB Custom Calculation: 0.3818 seconds


,WhiteElo,BlackElo,EloDifference
0,1901,1896,5
1,1641,1627,14
2,1647,1688,41
3,1706,1317,389
4,1945,1900,45


In [46]:
import pandas as pd
from IPython.display import display

# Define the execution times
results_data = {
    "Tool": ["Pandas", "DuckDB"],
    "Execution Time (seconds)": [pandas_custom_time, duckdb_custom_time]  # Pandas exceeds time limit
}

# Create a DataFrame for display
comparison_table = pd.DataFrame(results_data)

# Display the comparison table
display(comparison_table)


,Tool,Execution Time (seconds)
0,Pandas,0.199117
1,DuckDB,0.381771


### Why Did DuckDB Take More Time for the Custom Calculation?

While DuckDB is optimized for large-scale analytical tasks, the custom calculation scenario demonstrates where **Pandas may have an advantage** for small, Python-native operations.

#### **Key Reasons for DuckDB's Slower Performance**
1. **SQL Query Overhead**:
   - DuckDB requires parsing and optimizing the SQL query before execution.
   - For smaller datasets or simple operations, this overhead can make DuckDB slightly slower compared to Pandas.

2. **Data Handling Differences**:
   - In DuckDB, the query operates on the registered dataset (`df`), and the result is fetched as a new DataFrame.
   - In Pandas, the operation is performed directly on the in-memory DataFrame, avoiding extra processing steps.

3. **Python-Native Advantage in Pandas**:
   - The calculation (`abs(df['WhiteElo'] - df['BlackElo'])`) is a simple, vectorized operation in Pandas.
   - Pandas leverages NumPy's highly optimized array operations, which are extremely fast for small to medium-sized data.


# Applications and Conclusion

### Applications
- DuckDB excels when:
  - The dataset is **large** (millions of rows).
  - The operation involves **complex queries** (e.g., filtering, joins, aggregations).
  - Data is stored in columnar formats like Parquet or Arrow.

- Pandas is best for:
  - Quick prototyping.
  - Small-to-medium datasets.

### Conclusion
- **Pandas** is more efficient for small-scale, Python-native operations like this custom calculation.
- **DuckDB** shines in handling large datasets and complex analytical queries where its optimizations become more impactful.